Agenda
1. preparation
2. Model

In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import gc
import re
from collections import Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
from tqdm.auto import tqdm
tqdm.pandas()

from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from xgboost import XGBRegressor

# 1-1. load data

In [2]:
train_logs = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
display(train_logs)
train_scores = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')
display(train_scores)
test_logs = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')
display(test_logs)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
...,...,...,...,...,...,...,...,...,...,...,...
8405893,fff05981,3615,2063944,2064440,496,Nonproduction,Leftclick,Leftclick,NoChange,1031,240
8405894,fff05981,3616,2064497,2064497,0,Nonproduction,Shift,Shift,NoChange,1031,240
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240


,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0
...,...,...
2466,ffb8c745,3.5
2467,ffbef7e5,4.0
2468,ffccd6fd,1.5
2469,ffec5b38,5.0


,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0
5,4444cccc,2,184996,185052,56,Input,q,q,q,1,1


## 1-2. helper functions

### 1-2-1. const definition

In [3]:
# variables category
exp_key_column = "id"
obj_column = "score"

In [4]:
# count_elements
target_activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
target_events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
          'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
target_text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
target_punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']

In [5]:
def element_counts(df: pd.DataFrame, key_colname: str, count_colname: str, count_elements: list, match: str):
    """
    This function counts each elements
    Parameters
    ----------
    df: DataFrame
    key_colname: aggregator
    count_colname: count columns
    count_elements: count value in this list
    match: exact or partital
        
    Returns
    -------
    d : pd.DataFrame
    """
    print(f"======================element_counts start({count_colname})!!======================")
    tmp_df = df.groupby(key_colname).agg({count_colname: list}).reset_index()
    display(tmp_df)
    ret = list()
    if match == "exact":
        for li in tqdm(tmp_df[count_colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in count_elements:
                di[k] = 0
            for item in items:
                k,v = item[0], item[1]
                if k in di: # this part counts disgnated part
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        display(ret)
        cols = [f"{count_colname}_{i}_count" for i in range(len(ret.columns))]
        ret.columns = cols
    elif match == "partitial":
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in count_elements:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        
    return ret
    
def get_input_words(df: pd.DataFrame):
    print("======================get_input_words start!!======================")
    tmp_df = df[(~df["text_change"].str.contains("=>"))&(df["text_change"] != "Nochange")].reset_index(drop= True)
    tmp_df = tmp_df.groupby(exp_key_column).agg({"text_change": list}).reset_index()
    # concat part
    tmp_df["text_change"] = tmp_df["text_change"].apply(lambda x: "".join(x))
    tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
    
    tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
    display(tmp_df[["id","text_change"]])
    
    # calc part
    tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df.drop(['text_change'], axis=1, inplace=True)
    return tmp_df

## 1-3. make features

In [6]:
def make_feats(df: pd.DataFrame):
    print("======================make_feats start!!======================")
    feats = pd.DataFrame({exp_key_column: df[exp_key_column].unique().tolist()})
    # time shift
    target_col = "up_time"
    df[f'{target_col}_shift1'] = df.groupby(exp_key_column)[target_col].shift(1) #(going under ↓)
    df['action_time_gap'] = df['down_time'] - df[f'{target_col}_shift1']
    df.drop(f'{target_col}_shift1', axis=1, inplace=True)
    
    # cursor position shift
    target_col = "cursor_position"
    df[f'{target_col}_shift1'] = df.groupby(exp_key_column)[target_col].shift(1)
    df[f'{target_col}_change'] = np.abs(df[target_col] - df[f'{target_col}_shift1'])
    df.drop(f'{target_col}_shift1', axis=1, inplace=True)
    
    # word count shift
    target_col = "word_count"
    df[f'{target_col}_shift1'] = df.groupby(exp_key_column)[target_col].shift(1)
    df[f'{target_col}_change'] = np.abs(df[target_col] - df[f'{target_col}_shift1'])
    df.drop(f'{target_col}_shift1', axis=1, inplace=True)
    
    for item in tqdm([
        ('event_id', ['max']),
        ('up_time', ['max']),
        ('action_time', ['sum', 'max', 'mean', 'std']),
        ('activity', ['nunique']),
        ('down_event', ['nunique']),
        ('up_event', ['nunique']),
        ('text_change', ['nunique']),
        ('cursor_position', ['nunique', 'max', 'mean']),
        ('word_count', ['nunique', 'max', 'mean']),
        ('action_time_gap', ['max', 'min', 'mean', 'std', 'sum']),
        ('cursor_position_change', ['max', 'mean', 'std', 'sum']),
        ('word_count_change', ['max', 'mean', 'std', 'sum'])
    ]):
        colname, methods = item[0], item[1]
        for method in methods:
            print(f"calc: {method} of {colname}")
            tmp_df = df.groupby([exp_key_column]).agg({colname: method}).reset_index().rename(columns = {colname: f"{colname}_{method}"})
            feats = feats.merge(tmp_df, on = exp_key_column, how= "left")
    
    # counts
#     element_counts(df: pd.DataFrame, key_colname = exp_key_column: str, count_colname: str, count_elements: list, match: str):
    # tmp_df = activity_counts(df)
    tmp_df = element_counts(df, exp_key_column, "activity", target_activities, "exact")
    feats = pd.concat([feats, tmp_df], axis=1)
    
#     tmp_df = event_counts(df, 'down_event')
    tmp_df = element_counts(df, exp_key_column, "down_event", target_events, "exact")
    feats = pd.concat([feats, tmp_df], axis=1)
    
#     tmp_df = event_counts(df, 'up_event')
    tmp_df = element_counts(df, exp_key_column, "up_event", target_events, "exact")
    feats = pd.concat([feats, tmp_df], axis=1)
    
#     tmp_df = text_change_counts(df)
    tmp_df = element_counts(df, exp_key_column, "text_change", target_text_changes, "exact")
    feats = pd.concat([feats, tmp_df], axis=1)
    
#     tmp_df = match_punctuations(df)
    tmp_df = element_counts(df, exp_key_column, "down_event", target_punctuations, "partitial")
    feats = pd.concat([feats, tmp_df], axis=1)
    
    
    # input words
    tmp_df = get_input_words(df)
    feats = pd.merge(feats, tmp_df, on='id', how='left')
    
    # compare feats
    feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
    feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
    feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
    feats['idle_time_ratio'] = feats['action_time_gap_sum'] / feats['up_time_max']
    
    return feats
    

In [7]:
train_feats = make_feats(train_logs)

======================make_feats start!!======================


  0%|          | 0/12 [00:00<?, ?it/s]

calc: max of event_id
calc: max of up_time
calc: sum of action_time
calc: max of action_time
calc: mean of action_time
calc: std of action_time
calc: nunique of activity
calc: nunique of down_event
calc: nunique of up_event
calc: nunique of text_change
calc: nunique of cursor_position
calc: max of cursor_position
calc: mean of cursor_position
calc: nunique of word_count
calc: max of word_count
calc: mean of word_count
calc: max of action_time_gap
calc: min of action_time_gap
calc: mean of action_time_gap
calc: std of action_time_gap
calc: sum of action_time_gap
calc: max of cursor_position_change
calc: mean of cursor_position_change
calc: std of cursor_position_change
calc: sum of cursor_position_change
calc: max of word_count_change
calc: mean of word_count_change
calc: std of word_count_change
calc: sum of word_count_change
======================element_counts start(activity)!!======================


,id,activity
0,001519c8,"[Nonproduction, Nonproduction, Nonproduction, ..."
1,0022f953,"[Nonproduction, Nonproduction, Input, Input, I..."
2,0042269b,"[Nonproduction, Nonproduction, Input, Input, I..."
3,0059420b,"[Nonproduction, Nonproduction, Nonproduction, ..."
4,0075873a,"[Nonproduction, Nonproduction, Input, Input, I..."
...,...,...
2466,ffb8c745,"[Nonproduction, Nonproduction, Nonproduction, ..."
2467,ffbef7e5,"[Nonproduction, Nonproduction, Nonproduction, ..."
2468,ffccd6fd,"[Nonproduction, Nonproduction, Input, Input, I..."
2469,ffec5b38,"[Nonproduction, Nonproduction, Input, Input, I..."


  0%|          | 0/2471 [00:00<?, ?it/s]

...

======================element_counts start(down_event)!!======================


,id,down_event
0,001519c8,"[Leftclick, Leftclick, Shift, q, q, q, q, q, q..."
1,0022f953,"[Leftclick, Shift, q, q, q, q, Space, q, q, Sp..."
2,0042269b,"[Leftclick, Shift, q, q, q, q, q, q, q, Space,..."
3,0059420b,"[Leftclick, Leftclick, Shift, Shift, Shift, Sh..."
4,0075873a,"[Leftclick, Shift, q, q, q, q, q, q, q, q, q, ..."
...,...,...
2466,ffb8c745,"[Leftclick, Tab, Leftclick, Space, Space, Spac..."
2467,ffbef7e5,"[Leftclick, Leftclick, Shift, q, q, q, q, Spac..."
2468,ffccd6fd,"[Leftclick, Leftclick, q, q, q, q, q, q, Space..."
2469,ffec5b38,"[Leftclick, Shift, q, q, q, q, q, q, q, q, Bac..."


  0%|          | 0/2471 [00:00<?, ?it/s]

======================get_input_words start!!======================


,id,text_change
0,001519c8,"[qqqqqq, qqq, qqqqqqq, qqqqqq, qq, qqqq, qqqqq..."
1,0022f953,"[qqqq, qq, qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq, ..."
2,0042269b,"[qqqqqqq, qqq, qqqq, qqqq, qqqq, qq, qqqqqqqq,..."
3,0059420b,"[qqqq, qq, qqqqqqq, qqqqqq, q, q, qqq, qqqqqqq..."
4,0075873a,"[qqqqqqqqqqq, qq, q, qq, qqqqq, qq, qqqqqqqqqq..."
...,...,...
2466,ffb8c745,"[qq, qqqqq, q, qqqqqqq, qqq, q, qqqqq, q, qqqq..."
2467,ffbef7e5,"[qqqq, qqqqqq, qqqqq, qq, qqqqq, qqqqq, qq, qq..."
2468,ffccd6fd,"[qqqqqq, qqqq, q, qqqqqqq, qqqqqqqq, q, qq, qq..."
2469,ffec5b38,"[qqqqqqqqqqqq, qqqqqqq, qqqqqq, qqqq, qqqqq, q..."


In [8]:
test_feats = make_feats(test_logs)

======================make_feats start!!======================


  0%|          | 0/12 [00:00<?, ?it/s]

calc: max of event_id
calc: max of up_time
calc: sum of action_time
calc: max of action_time
calc: mean of action_time
calc: std of action_time
calc: nunique of activity
calc: nunique of down_event
calc: nunique of up_event
calc: nunique of text_change
calc: nunique of cursor_position
calc: max of cursor_position
calc: mean of cursor_position
calc: nunique of word_count
calc: max of word_count
calc: mean of word_count
calc: max of action_time_gap
calc: min of action_time_gap
calc: mean of action_time_gap
calc: std of action_time_gap
calc: sum of action_time_gap
calc: max of cursor_position_change
calc: mean of cursor_position_change
calc: std of cursor_position_change
calc: sum of cursor_position_change
calc: max of word_count_change
calc: mean of word_count_change
calc: std of word_count_change
calc: sum of word_count_change
======================element_counts start(activity)!!======================


,id,activity
0,0000aaaa,"[Input, Input]"
1,2222bbbb,"[Input, Input]"
2,4444cccc,"[Input, Input]"


  0%|          | 0/3 [00:00<?, ?it/s]

...

======================element_counts start(down_event)!!======================


,id,down_event
0,0000aaaa,"[Space, Space]"
1,2222bbbb,"[q, q]"
2,4444cccc,"[Space, q]"


  0%|          | 0/3 [00:00<?, ?it/s]

======================get_input_words start!!======================


,id,text_change
0,0000aaaa,[]
1,2222bbbb,[qq]
2,4444cccc,[q]


In [9]:
train_feats = train_feats.merge(train_scores, on='id', how='left')

In [10]:
display(train_feats)
display(test_feats)

,id,event_id_max,up_time_max,action_time_sum,action_time_max,action_time_mean,action_time_std,activity_nunique,down_event_nunique,up_event_nunique,text_change_nunique,cursor_position_nunique,cursor_position_max,cursor_position_mean,word_count_nunique,word_count_max,word_count_mean,action_time_gap_max,action_time_gap_min,action_time_gap_mean,action_time_gap_std,action_time_gap_sum,cursor_position_change_max,cursor_position_change_mean,cursor_position_change_std,cursor_position_change_sum,word_count_change_max,word_count_change_mean,word_count_change_std,word_count_change_sum,activity_0_count,activity_1_count,activity_2_count,activity_3_count,activity_4_count,down_event_0_count,down_event_1_count,down_event_2_count,down_event_3_count,down_event_4_count,down_event_5_count,down_event_6_count,down_event_7_count,down_event_8_count,down_event_9_count,down_event_10_count,down_event_11_count,down_event_12_count,down_event_13_count,down_event_14_count,down_event_15_count,up_event_0_count,up_event_1_count,up_event_2_count,up_event_3_count,up_event_4_count,up_event_5_count,up_event_6_count,up_event_7_count,up_event_8_count,up_event_9_count,up_event_10_count,up_event_11_count,up_event_12_count,up_event_13_count,up_event_14_count,up_event_15_count,text_change_0_count,text_change_1_count,text_change_2_count,text_change_3_count,text_change_4_count,text_change_5_count,text_change_6_count,text_change_7_count,text_change_8_count,text_change_9_count,text_change_10_count,text_change_11_count,text_change_12_count,text_change_13_count,text_change_14_count,punct_cnt,input_word_count,input_word_length_mean,input_word_length_max,input_word_length_std,word_time_ratio,word_event_ratio,event_time_ratio,idle_time_ratio,score
0,001519c8,2557,1801969,297243,2259,116.246774,91.797374,7,12,12,17,1469,1539,711.163473,257,256,128.116152,154136.0,-142.0,586.932707,4294.022274,1500200.0,1350.0,4.159624,43.180116,10632.0,2.0,0.172535,0.381013,441.0,2010,417,120,7,0,1619,357,417,27,2,92,2,21,12,0,0,4,0,3,0,0,1619,357,417,27,2,92,2,21,12,0,0,4,0,3,0,0,1940,436,120,28,14,4,5,0,0,0,1,0,0,0,0,37,377,5.169761,20,3.346931,0.000142,0.100117,0.001419,0.832534,3.5
1,0022f953,2454,1788969,275391,1758,112.221271,55.431189,5,17,17,12,1416,1676,776.205786,324,323,182.714751,145899.0,-166.0,604.547493,4897.303641,1482955.0,1581.0,9.819405,84.785626,24087.0,1.0,0.170404,0.376064,418.0,1938,260,254,1,1,1490,391,260,97,46,56,49,15,21,3,2,6,0,3,0,0,1490,391,260,97,46,56,49,15,21,3,2,6,0,3,0,0,1698,432,254,18,24,7,4,6,6,3,0,0,0,0,0,53,401,4.234414,33,3.062917,0.000181,0.131622,0.001372,0.828944,3.5
2,0042269b,4136,1771669,421201,3005,101.837766,82.383766,4,13,18,19,1649,2291,731.611702,405,404,194.772727,153886.0,-250.0,325.520435,3937.359025,1346027.0,1862.0,6.531318,71.786451,27007.0,28.0,0.167836,0.644564,694.0,3515,439,175,7,0,2904,552,439,39,6,129,0,21,23,0,0,17,0,0,0,0,2899,552,439,39,6,129,0,21,23,0,0,17,0,0,0,0,3257,615,175,23,26,23,0,2,1,0,0,4,0,0,0,47,639,5.344288,25,3.372135,0.000228,0.097679,0.002335,0.759751,6.0
3,0059420b,1556,1404469,189596,806,121.848329,113.768226,5,15,15,10,1048,1047,542.537275,207,206,103.618895,101690.0,-516.0,754.648232,4242.152639,1173478.0,357.0,1.457878,9.920533,2267.0,1.0,0.181350,0.385432,282.0,1304,151,99,1,1,1038,243,152,68,0,18,0,13,3,0,0,3,2,2,0,0,1038,243,152,68,0,18,0,13,3,0,0,3,2,2,0,0,1146,281,99,13,3,4,3,0,0,0,0,5,0,0,0,18,255,4.537255,15,2.867940,0.000147,0.132391,0.001108,0.835531,2.0
4,0075873a,2531,1662472,313702,701,123.943896,62.082013,3,11,11,9,1197,1402,600.050968,253,252,125.082971,110688.0,-158.0,502.094862,3896.209237,1270300.0,643.0,2.803953,24.251326,7094.0,1.0,0.168379,0.374277,426.0,1942,517,72,0,0,1541,324,517,39,0,33,0,23,24,0,0,10,0,17,0,0,1541,324,517,39,0,33,0,23,24,0,0,10,0,17,0,0,1964,397,72,32,25,12,25,0,0,2,0,2,0,0,0,66,431,4.556845,14,2.783927,0.000152,0.099565,0.001522,0.764103,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

,id,event_id_max,up_time_max,action_time_sum,action_time_max,action_time_mean,action_time_std,activity_nunique,down_event_nunique,up_event_nunique,text_change_nunique,cursor_position_nunique,cursor_position_max,cursor_position_mean,word_count_nunique,word_count_max,word_count_mean,action_time_gap_max,action_time_gap_min,action_time_gap_mean,action_time_gap_std,action_time_gap_sum,cursor_position_change_max,cursor_position_change_mean,cursor_position_change_std,cursor_position_change_sum,word_count_change_max,word_count_change_mean,word_count_change_std,word_count_change_sum,activity_0_count,activity_1_count,activity_2_count,activity_3_count,activity_4_count,down_event_0_count,down_event_1_count,down_event_2_count,down_event_3_count,down_event_4_count,down_event_5_count,down_event_6_count,down_event_7_count,down_event_8_count,down_event_9_count,down_event_10_count,down_event_11_count,down_event_12_count,down_event_13_count,down_event_14_count,down_event_15_count,up_event_0_count,up_event_1_count,up_event_2_count,up_event_3_count,up_event_4_count,up_event_5_count,up_event_6_count,up_event_7_count,up_event_8_count,up_event_9_count,up_event_10_count,up_event_11_count,up_event_12_count,up_event_13_count,up_event_14_count,up_event_15_count,text_change_0_count,text_change_1_count,text_change_2_count,text_change_3_count,text_change_4_count,text_change_5_count,text_change_6_count,text_change_7_count,text_change_8_count,text_change_9_count,text_change_10_count,text_change_11_count,text_change_12_count,text_change_13_count,text_change_14_count,punct_cnt,input_word_count,input_word_length_mean,input_word_length_max,input_word_length_std,word_time_ratio,word_event_ratio,event_time_ratio,idle_time_ratio
0,0000aaaa,2,760160,172,87,86.0,1.414214,1,1,1,1,2,1,0.5,1,0,0.0,421555.0,421555.0,421555.0,NaN,421555.0,1.0,1.0,NaN,1.0,0.0,0.0,NaN,0.0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0.000000,0.0,0.000003,0.554561
1,2222bbbb,2,712023,113,67,56.5,14.849242,1,1,1,1,2,1,0.5,1,1,1.0,-421521.0,-421521.0,-421521.0,NaN,-421521.0,1.0,1.0,NaN,1.0,0.0,0.0,NaN,0.0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.0,2,0.0,0.000001,0.5,0.000003,-0.592005
2,4444cccc,2,635641,150,94,75.0,26.870058,1,2,2,2,2,1,0.5,2,1,0.5,-450645.0,-450645.0,-450645.0,NaN,-450645.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,2,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,1,0.0,0.000002,0.5,0.000003,-0.708962


# 2. Model

## 2-1. prepartation

In [11]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [12]:
Y = train_feats[obj_column]
X = train_feats.drop([exp_key_column , obj_column] , axis=1)

In [13]:
metric = "rsme"

## 2-2. AutoML

In [14]:
!pip install TPOT

In [15]:
from tpot import TPOTRegressor

In [16]:
tpot = TPOTRegressor(scoring='neg_mean_absolute_error',
                  max_time_mins = 100,
                 generations=10,
                 population_size=2,
                 random_state=42,
                 verbosity=2,
                 n_jobs=-1,
                 memory = "auto"
                )

In [17]:
tpot.fit(X,Y)
tpot.fitted_pipeline_

Optimization Progress:   0%|          | 0/2 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.509503402687533

Generation 2 - Current best internal CV score: -0.5074745193145818

Generation 3 - Current best internal CV score: -0.5074745193145818

Generation 4 - Current best internal CV score: -0.5074745193145818

Generation 5 - Current best internal CV score: -0.5074745193145818

Generation 6 - Current best internal CV score: -0.5074745193145818

Generation 7 - Current best internal CV score: -0.5074745193145818

Generation 8 - Current best internal CV score: -0.5074745193145818

Generation 9 - Current best internal CV score: -0.5074745193145818

Generation 10 - Current best internal CV score: -0.5049753213604242

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=9, n_estimators=100)


Pipeline(memory=Memory(location=/tmp/tmp_5mwf25j/joblib),
         steps=[('randomforestregressor',
                 RandomForestRegressor(max_features=0.15000000000000002,
                                       min_samples_leaf=2, min_samples_split=9,
                                       random_state=42))])

In [18]:
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = np.sqrt(mean_squared_error(true, pred))
    return scores

scores = calculate_scores(Y, tpot.predict(X))
print(scores)

0.35668376302092397


## 2-2. study by Optuna
find best parameters of each models

In [19]:
models = {}
n_trials = 100
n_splits = 10

### 2-2-1. lgb

In [20]:
def lgb_objective(trial,data=X,target=Y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param = {
        'metric': 'rmse', 
        'random_state': 42,
        'n_estimators': trial.suggest_int('n_estimators', 1000,30000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_int('max_depth', 1 , 100),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = lgb.LGBMRegressor(**param)  
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [21]:
lgb_dict = {}
lgb_study = optuna.create_study(direction='minimize')
lgb_study.optimize(lgb_objective, n_trials=n_trials)
print('Number of finished trials:', len(lgb_study.trials))
print('Best trial:', lgb_study.best_trial.params)
lgb_param = lgb_study.best_params 
lgb_dict["model"] = lgb.LGBMRegressor(**lgb_param)  

[I 2023-10-09 12:08:01,786] A new study created in memory with name: no-name-4383bb31-1df4-4722-b2c6-1714f5ba676f
[I 2023-10-09 12:08:04,761] Trial 0 finished with value: 0.5940865385360413 and parameters: {'n_estimators': 26374, 'reg_alpha': 0.2238569757746858, 'reg_lambda': 9.083694023094594, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.014, 'max_depth': 70, 'num_leaves': 524, 'min_child_samples': 110, 'min_data_per_groups': 71}. Best is trial 0 with value: 0.5940865385360413.
[I 2023-10-09 12:08:09,987] Trial 1 finished with value: 0.5904690537393543 and parameters: {'n_estimators': 10945, 'reg_alpha': 0.09715107908520595, 'reg_lambda': 0.008008998773501471, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.017, 'max_depth': 93, 'num_leaves': 374, 'min_child_samples': 38, 'min_data_per_groups': 80}. Best is trial 1 with value: 0.5904690537393543.
[I 2023-10-09 12:08:14,804] Trial 2 finished with value: 0.5876405148455026 and parameters: {'n_estimators': 

Number of finished trials: 100
Best trial: {'n_estimators': 5198, 'reg_alpha': 0.09506504872546441, 'reg_lambda': 0.3878016874169255, 'colsample_bytree': 0.3, 'subsample': 0.7, 'learning_rate': 0.006, 'max_depth': 92, 'num_leaves': 473, 'min_child_samples': 69, 'min_data_per_groups': 60}


### 2-2-2. xgb

In [22]:
def xgb_objective(trial,data=X,target=Y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [23]:
xgb_dict = {}
xgb_study = optuna.create_study(direction='minimize')
xgb_study.optimize(xgb_objective, n_trials=n_trials)
print('Number of finished trials:', len(xgb_study.trials))
print('Best trial:', xgb_study.best_trial.params)
xgb_params=xgb_study.best_params 
xgb_dict["model"] = XGBRegressor(**xgb_params)

[I 2023-10-09 12:17:45,628] A new study created in memory with name: no-name-ad974c2a-98e1-4a9a-85f4-fddecae02bcf
[I 2023-10-09 12:17:46,252] Trial 0 finished with value: 0.6076945255167925 and parameters: {'max_depth': 3, 'learning_rate': 0.2417530766428139, 'n_estimators': 403, 'min_child_weight': 4, 'gamma': 0.3243433360228129, 'subsample': 0.5395638344392585, 'colsample_bytree': 0.5847142221492413, 'reg_alpha': 0.11768296835935409, 'reg_lambda': 0.3915295618953976, 'random_state': 689}. Best is trial 0 with value: 0.6076945255167925.
[I 2023-10-09 12:17:46,374] Trial 1 finished with value: 0.8452239419924465 and parameters: {'max_depth': 4, 'learning_rate': 0.8389960463961145, 'n_estimators': 145, 'min_child_weight': 5, 'gamma': 0.20279634035746472, 'subsample': 0.21215202564657576, 'colsample_bytree': 0.019431742626764156, 'reg_alpha': 0.4406624474262899, 'reg_lambda': 0.7489231875921469, 'random_state': 927}. Best is trial 0 with value: 0.6076945255167925.
[I 2023-10-09 12:17:47,

Number of finished trials: 100
Best trial: {'max_depth': 3, 'learning_rate': 0.029059302771127754, 'n_estimators': 756, 'min_child_weight': 7, 'gamma': 0.1288426226034181, 'subsample': 0.34151292146926276, 'colsample_bytree': 0.6265844011039098, 'reg_alpha': 0.5127042069723139, 'reg_lambda': 0.6444726358596109, 'random_state': 717}


In [24]:
models["lgb"] = lgb_dict
models["xgb"] = xgb_dict

## 2-3. train lGBM

In [25]:
feature_names = list(filter(lambda x: x not in [obj_column, exp_key_column], train_feats.columns))

In [26]:
oof = []
prediction = test_feats[[exp_key_column]]
prediction[obj_column] = 0
df_importance_list = []

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train_feats[feature_names])):
    X_train = train_feats.iloc[trn_idx][feature_names]
    Y_train = train_feats.iloc[trn_idx][obj_column]

    X_val = train_feats.iloc[val_idx][feature_names]
    Y_val = train_feats.iloc[val_idx][obj_column]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))
    for key, value in models.items():
        print(f"***********{key}***************")
        best_model = value["model"]
        
        ## we have to change because each models have different params
        if key == "xgb":
            model = best_model.fit(X_train,
                                  Y_train,
                                  # eval_names=['train', 'valid'],
                                  eval_set=[(X_train, Y_train), (X_val, Y_val)],
                                  verbose=20,
                                  # eval_metric= [metric],
                                  early_stopping_rounds=50)
            pred_val = model.predict(X_val)
            pred_test = model.predict(test_feats[feature_names])
            prediction[obj_column] += pred_test / kfold.n_splits
        elif key == "lgb":
            model = best_model.fit(X_train,
                      Y_train,
                      eval_names=['train', 'valid'],
                      eval_set=[(X_train, Y_train), (X_val, Y_val)],
                      verbose=20,
                      eval_metric= metric,
                      early_stopping_rounds=50)
            pred_val = model.predict(X_val, num_iteration=model.best_iteration_)
            pred_test = model.predict(test_feats[feature_names], num_iteration=model.best_iteration_)
            prediction[obj_column] += pred_test / kfold.n_splits
        
        df_oof = train_feats.iloc[val_idx][[exp_key_column, obj_column]].copy()
        df_oof['pred'] = pred_val
        oof.append(df_oof)

              
        models[key]["prediction"] = prediction

        df_importance = pd.DataFrame({
            'column': feature_names,
            'importance': model.feature_importances_,
        })
        df_importance_list.append(df_importance)

        del model, pred_val, pred_test
        gc.collect()
        df_oof = pd.concat(oof)
        rmse = mean_squared_error(df_oof[obj_column], np.clip(df_oof['pred'], a_min=0.5, a_max=6.0), squared=False)
        print('rmse:', rmse)
        models[key]["score"] = rmse


Fold_1 Training ================================

***********lgb***************
[LightGBM] [Warning] Unknown parameter: min_data_per_groups
[20]	train's l2: 0.922031	valid's l2: 0.91654
[40]	train's l2: 0.817199	valid's l2: 0.820129
[60]	train's l2: 0.733199	valid's l2: 0.742587
[80]	train's l2: 0.663898	valid's l2: 0.680343
[100]	train's l2: 0.608288	valid's l2: 0.630306
[120]	train's l2: 0.562619	valid's l2: 0.589851
[140]	train's l2: 0.524638	valid's l2: 0.55689
[160]	train's l2: 0.493471	valid's l2: 0.530124
[180]	train's l2: 0.467393	valid's l2: 0.508063
[200]	train's l2: 0.445324	valid's l2: 0.489253
[220]	train's l2: 0.426487	valid's l2: 0.4743
[240]	train's l2: 0.410222	valid's l2: 0.461507
[260]	train's l2: 0.395775	valid's l2: 0.450137
[280]	train's l2: 0.382815	valid's l2: 0.439573
[300]	train's l2: 0.371506	valid's l2: 0.431094
[320]	train's l2: 0.36176	valid's l2: 0.424491
[340]	train's l2: 0.352715	valid's l2: 0.418608
[360]	train's l2: 0.344283	valid's l2: 0.413426
[380

In [27]:
for key, value in models.items():
    print(key)
    print(value["score"])

lgb
0.6403559859823206
xgb
0.6381749792367313


# 3. submission

In [28]:
display(prediction)
prediction['score'] = np.clip(prediction['score'], a_min=0.5, a_max=6.0)
prediction.to_csv('submission.csv', index=False)

,id,score
0,0000aaaa,2.522412
1,2222bbbb,2.434571
2,4444cccc,2.385136
